In [10]:
import os
import pandas as pd
import glob
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
import concurrent.futures 
from sklearn.metrics import accuracy_score
import requests
from bs4 import BeautifulSoup
import time
import random
from fuzzywuzzy import fuzz, process
from os import walk
from datetime import datetime
import numpy as np

In [26]:
df = pd.read_csv('pred_profits.csv', index_col = [0])

In [8]:
dha = {'draw':'0','home':'1','away':'2'}

In [29]:
# run to test current profits based on the newest data
df = df.replace('no value', 0)
df[['draw','home','away']] = df[['draw','home','away']].astype(float)
for word, number in dha.items():
    df[f'{word}_profits'] = np.where(df['final_score'] == int(number),
                                    df[word] * (df[number] - 1) * 100,
                                    -100*df[word])

df['profits'] = df[df.columns[-3:]].sum(axis = 1)
p = round(np.sum(df["profits"]),3)
df = df.round(3)
df.to_csv(f'profits_{datetime.now():%Y%m%d}.csv')
print(f'number of bets made: {df.shape[0]}')
print(f'profit on $100 bankroll: ${p}')

number of bets made: 74
profit on $100 bankroll: $-384.637


In [30]:
df

,0,1,2,final_score,Label,Score_0,Score_1,Score_2,draw,home,away,draw_profits,home_profits,away_profits,profits
0,2.300,3.100,3.200,0,0,0.971,0.010,0.019,0.949,0.000,0.0,123.422,-0.000,-0.0,123.422
1,4.600,1.293,13.000,1,1,0.044,0.956,0.000,0.000,0.806,0.0,-0.000,23.598,-0.0,23.598
2,3.800,1.400,9.500,1,0,0.668,0.321,0.010,0.550,0.000,0.0,-54.980,0.000,-0.0,-54.980
3,2.625,2.050,5.125,1,1,0.249,0.617,0.134,0.000,0.252,0.0,-0.000,26.502,-0.0,26.502
4,2.500,4.550,2.150,2,0,0.984,0.005,0.012,0.972,0.000,0.0,-97.250,-0.000,0.0,-97.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,3.275,1.775,4.500,2,1,0.359,0.592,0.050,0.077,0.065,0.0,-7.710,-6.460,0.0,-14.170
70,5.900,1.167,19.000,1,1,0.047,0.952,0.001,0.000,0.668,0.0,-0.000,11.125,-0.0,11.125
71,2.375,2.550,3.800,0,0,0.887,0.104,0.009,0.805,0.000,0.0,110.715,-0.000,-0.0,110.715
72,3.100,3.400,2.165,2,0,0.811,0.155,0.034,0.721,0.000,0.0,-72.090,-0.000,0.0,-72.090


In [6]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [7]:
def get_winner(url):
    page = requests.get(url)
    time.sleep(random.randint(0, 4))
    soup = BeautifulSoup(page.content, 'html.parser')
    score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
    score = int(score[0]) - int(score[1])
    score = winner(score)
    return score

In [8]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [31]:
scores = []
for i in range(0, len(df['url']), 3):
    x = list(df['url'][i:i + 3])
    print(x)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        r = executor.map(get_winner, x)
    for result in r:
        scores.append(result)

['https://www.fotmob.com/match/3888550/matchfacts/puebla-vs-fc-juarez', 'https://www.fotmob.com/match/3888551/matchfacts/mazatlan-fc-vs-cf-america', 'https://www.fotmob.com/match/3900373/matchfacts/fc-utrecht-vs-ajax']
['https://www.fotmob.com/match/3904422/matchfacts/nantes-vs-toulouse', 'https://www.fotmob.com/match/3902828/matchfacts/dundee-united-vs-celtic', 'https://www.fotmob.com/match/3916965/matchfacts/gent-vs-royal-antwerp']
['https://www.fotmob.com/match/3887414/matchfacts/randers-fc-vs-aalborg-bk', 'https://www.fotmob.com/match/3887416/matchfacts/viborg-vs-lyngby', 'https://www.fotmob.com/match/3905446/matchfacts/lugano-vs-st.-gallen']
['https://www.fotmob.com/match/3900374/matchfacts/excelsior-vs-psv-eindhoven', 'https://www.fotmob.com/match/3900375/matchfacts/fc-volendam-vs-fc-twente', 'https://www.fotmob.com/match/3900963/matchfacts/aston-villa-vs-west-ham-united']
['https://www.fotmob.com/match/3900971/matchfacts/wolverhampton-wanderers-vs-newcastle-united', 'https://www

['https://www.fotmob.com/match/3888623/matchfacts/necaxa-vs-leon', 'https://www.fotmob.com/match/3900986/matchfacts/everton-vs-liverpool', 'https://www.fotmob.com/match/3902849/matchfacts/celtic-vs-rangers']
['https://www.fotmob.com/match/3917967/matchfacts/mallorca-vs-girona', 'https://www.fotmob.com/match/3919120/matchfacts/fiorentina-vs-juventus', 'https://www.fotmob.com/match/3903581/matchfacts/bayer-leverkusen-vs-sc-freiburg']
['https://www.fotmob.com/match/3903586/matchfacts/vfl-bochum-vs-werder-bremen', 'https://www.fotmob.com/match/3903582/matchfacts/union-berlin-vs-bayern-m%C3%BCnchen', 'https://www.fotmob.com/match/3903585/matchfacts/wolfsburg-vs-1.-fc-k%C3%B6ln']
['https://www.fotmob.com/match/3900983/matchfacts/brentford-vs-leeds-united', 'https://www.fotmob.com/match/3900985/matchfacts/chelsea-vs-west-ham-united', 'https://www.fotmob.com/match/3900988/matchfacts/newcastle-united-vs-crystal-palace']
['https://www.fotmob.com/match/3900990/matchfacts/tottenham-hotspur-vs-fulh